In [1]:
import pandas as pd
import numpy as np
import time
from geopy.geocoders import Nominatim
from tqdm import tqdm
geolocator = Nominatim(user_agent='giovani_valdrighi')

# Limpeza de dados

## Manténdo dados de São Paulo e colunas importantes

In [2]:
chunk_iter = pd.read_csv("data/MICRODADOS_ENEM_2019.csv", encoding = 'latin1', sep = ";", chunksize = 1000000)

In [3]:
total_rows = 0
chunks_filtered = []
for chunk in chunk_iter:
    total_rows += chunk.shape[0]
    chunks_filtered.append(chunk[chunk['SG_UF_RESIDENCIA'] == 'SP'])
df_sp = pd.concat(chunks_filtered)

In [4]:
print(f"Linhas totais: {total_rows}")
print(f"Colunas totais: {df_sp.shape[1]}")
print()
print("Mantendo apenas as linhas do estado de São Paulo")
print(f"Linhas restantes: {df_sp.shape[0]}")
print("Removendo colunas que não serão usadas:")
print("Colunas com informações sobre cada questão do teste")
print("TX_RESPOSTAS_CN   TX_RESPOSTAS_CH   TX_RESPOSTAS_LC   TX_RESPOSTAS_MT  \nTX_GABARITO_CN   TX_GABARITO_CH   TX_GABARITO_LC   TX_GABARITO_MT")
df_sp = df_sp.drop(columns = ['TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH','TX_RESPOSTAS_LC','TX_RESPOSTAS_MT',
                              'TX_GABARITO_CN','TX_GABARITO_CH','TX_GABARITO_LC','TX_GABARITO_MT'])
print("Colunas com códigos que não serão úteis:")
print('CO_MUNICIPIO_NASCIMENTO  CO_UF_NASCIMENTO   CO_MUNICIPIO_ESC   CO_UF_ESC\nCO_PROVA_CN   CO_PROVA_CH CO_PROVA_LC  CO_PROVA_MT')
df_sp = df_sp.drop(columns = ['CO_MUNICIPIO_NASCIMENTO', 'CO_UF_NASCIMENTO', 'CO_MUNICIPIO_ESC',
                              'CO_UF_ESC', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT'])
print(f"Colunas restantes: {df_sp.shape[1]}")

Linhas totais: 5095270
Colunas totais: 136

Mantendo apenas as linhas do estado de São Paulo
Linhas restantes: 813772
Removendo colunas que não serão usadas:
Colunas com informações sobre cada questão do teste
TX_RESPOSTAS_CN   TX_RESPOSTAS_CH   TX_RESPOSTAS_LC   TX_RESPOSTAS_MT  
TX_GABARITO_CN   TX_GABARITO_CH   TX_GABARITO_LC   TX_GABARITO_MT
Colunas com códigos que não serão úteis:
CO_MUNICIPIO_NASCIMENTO  CO_UF_NASCIMENTO   CO_MUNICIPIO_ESC   CO_UF_ESC
CO_PROVA_CN   CO_PROVA_CH CO_PROVA_LC  CO_PROVA_MT
Colunas restantes: 120


In [5]:
df_sp.to_csv("data/MICRODADOS_ENEM_SP_2019.csv", index = False)

## Limpeza para modelos

In [40]:
#keeping only the rows where the student went to the test
df = df_sp[(df_sp.TP_PRESENCA_CN == 1) & (df_sp.TP_PRESENCA_CH == 1) &
       (df_sp.TP_PRESENCA_LC == 1) & (df_sp.TP_PRESENCA_MT == 1)]

#removing presence columns
df = df.drop(columns = ['TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT'])

#creating NOTA column
df['NOTA'] = 0.2 * (df.NU_NOTA_CN + df.NU_NOTA_CH + df.NU_NOTA_MT + df.NU_NOTA_LC + df.NU_NOTA_REDACAO)

#removing other notes columns
df = df.drop(columns = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_MT', 'NU_NOTA_LC',
                        'NU_NOTA_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 
                        'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'TP_STATUS_REDACAO'])

#removing columns that don't have relation with the result in the test
df = df.drop(columns = ['NU_INSCRICAO', 'NU_ANO'])

#removing columns with spatial information
df = df.drop(columns = ['CO_MUNICIPIO_RESIDENCIA', 'NO_MUNICIPIO_RESIDENCIA', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA',
                        'NO_MUNICIPIO_NASCIMENTO', 'SG_UF_NASCIMENTO', 'CO_ESCOLA', 'NO_MUNICIPIO_ESC',
                        'SG_UF_ESC', 'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA', 'CO_UF_PROVA', 'SG_UF_PROVA',])



In [41]:
#renaming columns
columns = ['NUM_IDADE', 'CAT_SEXO', 'CAT_ESTADO_CIVIL', 'CAT_COR_RACA',
       'CAT_NACIONALIDADE', 'CAT_ST_CONCLUSAO', 'NUM_ANO_CONCLUIU', 'CAT_ESCOLA',
       'CAT_ENSINO', 'CAT_TREINEIRO', 'CAT_DEPENDENCIA_ADM_ESC',
       'CAT_LOCALIZACAO_ESC', 'CAT_SIT_FUNC_ESC', 'CAT_BAIXA_VISAO',
       'CAT_CEGUEIRA', 'CAT_SURDEZ', 'CAT_DEFICIENCIA_AUDITIVA',
       'CAT_SURDO_CEGUEIRA', 'CAT_DEFICIENCIA_FISICA', 'CAT_DEFICIENCIA_MENTAL',
       'CAT_DEFICIT_ATENCAO', 'CAT_DISLEXIA', 'CAT_DISCALCULIA', 'CAT_AUTISMO',
       'CAT_VISAO_MONOCULAR', 'CAT_OUTRA_DEF', 'CAT_GESTANTE', 'CAT_LACTANTE',
       'CAT_IDOSO', 'CAT_ESTUDA_CLASSE_HOSPITALAR', 'CAT_SEM_RECURSO',
       'CAT_BRAILLE', 'CAT_AMPLIADA_24', 'CAT_AMPLIADA_18', 'CAT_LEDOR',
       'CAT_ACESSO', 'CAT_TRANSCRICAO', 'CAT_LIBRAS', 'CAT_TEMPO_ADICIONAL',
       'CAT_LEITURA_LABIAL', 'CAT_MESA_CADEIRA_RODAS',
       'CAT_MESA_CADEIRA_SEPARADA', 'CAT_APOIO_PERNA', 'CAT_GUIA_INTERPRETE',
       'CAT_COMPUTADOR', 'CAT_CADEIRA_ESPECIAL', 'CAT_CADEIRA_CANHOTO',
       'CAT_CADEIRA_ACOLCHOADA', 'CAT_PROVA_DEITADO', 'CAT_MOBILIARIO_OBESO',
       'CAT_LAMINA_OVERLAY', 'CAT_PROTETOR_AURICULAR', 'CAT_MEDIDOR_GLICOSE',
       'CAT_MAQUINA_BRAILE', 'CAT_SOROBAN', 'CAT_MARCA_PASSO', 'CAT_SONDA',
       'CAT_MEDICAMENTOS', 'CAT_SALA_INDIVIDUAL', 'CAT_SALA_ESPECIAL',
       'CAT_SALA_ACOMPANHANTE', 'CAT_MOBILIARIO_ESPECIFICO',
       'CAT_MATERIAL_ESPECIFICO', 'CAT_NOME_SOCIAL', 'CAT_LINGUA', 'CAT_ESTUDO_PAI', 'CAT_ESTUDO_MAE',
       'CAT_PROFISSAO_PAI', 'CAT_PROFISSAO_MAE', 'NUM_PESSOAS_RESIDENCIA', 'CAT_RENDA', 
       'NUM_EMPREGADO_DOMESTICO', 'NUM_BANHEIRO', 'NUM_QUARTOS', 'NUM_CARRO', 'NUM_MOTO', 
       'NUM_GELADEIRA', 'NUM_FREEZER', 'NUM_MAQUINA_LAVAR', 'NUM_MAQUINA_SECAR', 
       'NUM_MICRO_ONDAS', 'NUM_LAVAR_LOUCA', 'CAT_ASPIRADOR_PO', 'NUM_TV', 'CAT_DVD',
       'CAT_TV_ASSINATURA', 'NUM_CELULAR', 'CAT_TELEFONE', 'NUM_COMPUTADOR', 'CAT_INTERNET', 'NUM_NOTA']
df.columns = columns

#changing values for columns
df.NUM_EMPREGADO_DOMESTICO = df.NUM_EMPREGADO_DOMESTICO.map({'A': 0., 'B':1., 'C': 3., 'D':5.})
for col in ['BANHEIRO', 'QUARTOS', 'CARRO', 'MOTO', 'GELADEIRA', 'FREEZER',
           'MAQUINA_LAVAR', 'MAQUINA_SECAR', 'MICRO_ONDAS', 'LAVAR_LOUCA', 
            'TV', 'CELULAR', 'COMPUTADOR']:
    df['NUM_'+col] = df['NUM_'+col].map({'A':0., 'B':1, 'C': 2, 'D':3, 'E':4})
    


In [42]:
print('Columns with NaNs.')
print(df.loc[:, (df.isna().mean() > 0)].isna().mean())
df.dropna().to_csv("data/ENEM_CLEAN.csv", index = False)

df = df.dropna(subset = ['NUM_IDADE'])
#replacing nans
for col in ['ENSINO', 'DEPENDENCIA_ADM_ESC',
           'LOCALIZACAO_ESC', 'SIT_FUNC_ESC']:
    df['CAT_'+col] = df['CAT_'+col].fillna(value = 'FALTANTE')

df.to_csv('data/ENEM_CLEAN_WITH_NAN.csv', index = False)

Columns with NaNs.
NUM_IDADE                  0.000022
CAT_ENSINO                 0.435017
CAT_DEPENDENCIA_ADM_ESC    0.695623
CAT_LOCALIZACAO_ESC        0.695623
CAT_SIT_FUNC_ESC           0.695623
dtype: float64
